In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
sd = drive.CreateFile({'id': '10rZZeiXHfqjIO4RorVLBB9TM0vhj6viB'})
sd.GetContentFile('nm_12_17_merged.csv')

In [0]:

from datetime import datetime
import pandas as pd
data = pd.read_csv('nm_12_17_merged.csv')
#d.head()

In [0]:
corr_data1 = data.drop(['state','id','newspaper','date','notice_id','combinations','Unnamed: 0', 'Unnamed: 0.1_x','Unnamed: 0.1_y','State_x'],axis=1)

In [5]:
corr_data1.columns

Index(['county', 'count_tags', 'combinations_clean', 'year', 'month', 'day',
       'FIPS', 'Rural_urban_continuum_code_2013', 'Urban_influence_code_2013',
       'Metro_2013', 'Median_Household_Income_2016',
       'Med_HH_Income_Percent_of_State_Total_2016',
       'Rural-urban_Continuum_Code_2003', 'Urban_Influence_Code_2003_x',
       'Rural-urban_Continuum_Code_2013', 'Urban_Influence_Code_2013_x',
       'POVALL_2016', 'POV017_2016', 'MEDHHINC_2016',
       '2013 Rural-urban Continuum Code', '2013 Urban Influence Code',
       'Less than a high school diploma, 2012-2016',
       'High school diploma only, 2012-2016',
       'Some college or associate's degree, 2012-2016',
       'Bachelor's degree or higher, 2012-2016',
       'Percent of adults with less than a high school diploma, 2012-2016',
       'Percent of adults with a high school diploma only, 2012-2016',
       'Percent of adults completing some college or associate's degree, 2012-2016',
       'Percent of adults with a

In [0]:
corr_data1['count']=corr_data1.groupby(['county','combinations_clean', 'year', 'month']).combinations_clean.transform('count')

In [0]:
temp = corr_data1.groupby(['county','combinations_clean', 'year', 'month','Unemployment_rate','R_DOMESTIC_MIG','Percent of adults with a bachelor\'s degree or higher, 2012-2016']).size().reset_index()

temp.rename(columns={0: 'count'}, inplace=True)

In [8]:
temp.columns

Index(['county', 'combinations_clean', 'year', 'month', 'Unemployment_rate',
       'R_DOMESTIC_MIG',
       'Percent of adults with a bachelor's degree or higher, 2012-2016',
       'count'],
      dtype='object')

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(temp['combinations_clean'])
le.transform(temp['combinations_clean'])
temp["combinations_clean_code"] = le.fit_transform(temp["combinations_clean"])

In [0]:
temp = temp[temp['combinations_clean'].str.contains("Foreclosure")]
# temp = temp[temp['combinations_clean'].isin(['1-Bids_2-Auctions_3-Property_4-Foreclosure',
#                                                            '1-Bids_2-Auctions_3-Property_4-Foreclosure_5-Default',
#                                                            '1-Summon_2-Foreclosure'])]

In [14]:
temp.shape

(1299, 9)

In [15]:
le1 = preprocessing.LabelEncoder()
le1.fit(temp['county'])
le1.transform(temp['county'])
temp["county_code"] = le.fit_transform(temp["county"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
temp.columns

Index(['county', 'combinations_clean', 'year', 'month', 'Unemployment_rate',
       'R_DOMESTIC_MIG',
       'Percent of adults with a bachelor's degree or higher, 2012-2016',
       'count', 'combinations_clean_code', 'county_code'],
      dtype='object')

In [0]:

train_x = temp[temp['year'] <2017]

In [18]:
train_x.shape

(1028, 10)

In [0]:
test_x = temp[temp['year'] == 2017]

In [0]:
train_x = train_x.drop(['combinations_clean','year','county'],axis=1)

In [21]:
train_x.head()

,month,Unemployment_rate,R_DOMESTIC_MIG,"Percent of adults with a bachelor's degree or higher, 2012-2016",count,combinations_clean_code,county_code
119,1,6.5,-3.543178,32.783,747,6,0
120,2,6.5,-3.543178,32.783,752,6,0
121,3,6.5,-3.543178,32.783,649,6,0
122,4,6.5,-3.543178,32.783,763,6,0
123,5,6.5,-3.543178,32.783,689,6,0


In [0]:
train_y = train_x['Unemployment_rate']

In [0]:
train_x = train_x.drop(['Unemployment_rate'],axis=1)

In [0]:
test_y = test_x['Unemployment_rate']

In [0]:
test_x = test_x.drop(['combinations_clean','year','county','Unemployment_rate'],axis=1)

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_x, train_y)
rf_predict = rf.predict(test_x)

In [27]:
test_y.shape

(271,)

In [28]:
rf_predict.shape

(271,)

In [29]:
rf.score(test_x,test_y)

-1.4945638775087482

In [30]:
from sklearn.metrics import explained_variance_score,mean_squared_log_error,mean_squared_error
explained_variance_score(test_y, rf_predict)

-0.8816343639201565

In [31]:
mean_squared_log_error(test_y, rf_predict)  

0.029761132208841762

In [32]:
mean_squared_error(test_y, rf_predict)  

1.7744361623616225

In [0]:
from sklearn import linear_model 
Model = linear_model.LinearRegression()
Model.fit(train_x, train_y)
modelResult = Model.predict(test_x)

In [34]:
mean_squared_error(test_y, modelResult) 

0.6602967165226205